In [2]:
# Import packages
import sys 
import os
sys.path.append(os.path.abspath("C:/Users/mieke/Documents/Msc Thesis/GitHub repositories/dicra/analytics/sandbox/notebooks/crop_yield_prediction/functions"))
import functions_time_series #import python file containing the functions
import functions_preprocessing #import python file containing the functions
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from statsmodels.tsa.seasonal import STL
import numpy as np
from sklearn.linear_model import LinearRegression
import datetime as dt
from matplotlib.colors import TwoSlopeNorm
import time
from rasterstats import zonal_stats
from scipy.stats import pearsonr
import plotly.express as px
import rasterio.mask
from osgeo import gdal
import re

In [3]:
# For updates
import importlib
importlib.reload(functions_time_series)
importlib.reload(functions_preprocessing)

<module 'functions_preprocessing_adjust' from 'c:\\Users\\mieke\\Documents\\Msc Thesis\\Notebooks Python\\Others\\functions_preprocessing_adjust.py'>

In [4]:
# Import this file to derive the polygon geometry for Telangana
bound_Telangana = gpd.read_file(r'C:\Users\mieke\Documents\Msc Thesis\GitHub repositories\dicra\src\data_preprocessing\base_geojson\TL_state_shapefile_for_clip.geojson')

In [5]:
# Import district boundaries
district_bounds = gpd.read_file('C:/Users/mieke/Documents/Msc Thesis/GitHub repositories/dicra/src/data_preprocessing/tsdm/District_Boundary.shp')
district_bounds = district_bounds.reset_index()
# Import mandal boundaries
mandal_bounds = gpd.read_file('C:/Users/mieke/Documents/Msc Thesis/GitHub repositories/dicra/src/data_preprocessing/tsdm/Mandal_Boundary.shp')
mandal_bounds = mandal_bounds.reset_index()

## Rescale Images

In [6]:
ndv = -9999

In [7]:
# Specified values for rescaling
offset = 0
scaling = 1 / 30

In [8]:
# For each of the LAI images, we crop the Geotiff file to Leaf Area Index in Telangana
path = 'C:/Users/mieke/Documents/Data_download/Copernicus/LAI/original/'
folder_names = os.listdir(path)
for i in folder_names:
    file_names = os.listdir(path + i)
    for j in file_names:
        LAI_data = path + i + "/" + j
        functions_preprocessing.rescale_unstable(j, LAI_data, ndv, scaling, offset, 'C:/Users/mieke/Documents/Data_download/Copernicus/LAI/Telangana/img_rescaled/', 'LAI_', [0,4,4,6,6,8])

In [14]:
# Specified values for rescaling SSM and SUSM
offset = 0
scaling = 1

In [11]:
# For each of the LAI images, we crop the Geotiff file to Leaf Area Index in Telangana
path = 'C:/Users/mieke/Documents/Data_download/GEE/SSM/GEE_tifs/'
file_names = os.listdir(path)
for j in file_names:
    file_path = path + j
    functions_preprocessing.rescale(j, file_path, ndv, scaling, offset, 'C:/Users/mieke/Documents/Data_download/GEE/SSM/Telangana/img_rescaled/', 'SSM_' + j[17:21] + '-' + j[21:23] + '-' + j[23:25] + '.tif')

In [15]:
# For each of the LAI images, we crop the Geotiff file to Leaf Area Index in Telangana
path = 'C:/Users/mieke/Documents/Data_download/GEE/SUSM/GEE_tifs/'
file_names = os.listdir(path)
for j in file_names:
    file_path = path + j
    functions_preprocessing.rescale(j, file_path, ndv, scaling, offset, 'C:/Users/mieke/Documents/Data_download/GEE/SUSM/Telangana/img_rescaled/', 'SUSM_' + j[17:21] + '-' + j[21:23] + '-' + j[23:25] + '.tif')

In [16]:
# Specified values for rescaling NDVI
offset = 0
scaling = 0.0001

In [17]:
# For each of the LAI images, we crop the Geotiff file to Leaf Area Index in Telangana
path = 'C:/Users/mieke/Documents/Data_download/GEE/NDVI/GEE_tifs/'
file_names = os.listdir(path)
for j in file_names:
    file_path = path + j
    functions_preprocessing.rescale(j, file_path, ndv, scaling, offset, 'C:/Users/mieke/Documents/Data_download/GEE/NDVI/Telangana/img_rescaled/', 'NDVI_' + j[0:4] + '-' + j[5:7] + '-' + j[8:10] + '.tif')

In [18]:
# Specified values for rescaling LST
offset = -273.15
scaling = 0.02

In [20]:
# For each of the LAI images, we crop the Geotiff file to Leaf Area Index in Telangana
path = 'C:/Users/mieke/Documents/Data_download/GEE/LST/GEE_tifs/'
file_names = os.listdir(path)
for j in file_names:
    file_path = path + j
    functions_preprocessing.rescale(j, file_path, ndv, scaling, offset, 'C:/Users/mieke/Documents/Data_download/GEE/LST/Telangana/img_rescaled/', 'LST_' + j[0:4] + '-' + j[5:7] + '-' + j[8:10] + '.tif')

## Crop Images

In [12]:
# For each of the LAI images, we rescale the Geotiff
path = 'C:/Users/mieke/Documents/Data_download/Copernicus/LAI/Telangana/img_rescaled/'
file_names = os.listdir(path) # contains all filenames
for  j in file_names:
    tiff_path = path + j # the selected GeoTIFF path
    functions_preprocessing.crop_image(j, tiff_path, bound_Telangana, 'C:/Users/mieke/Documents/Data_download/Copernicus/LAI/Telangana/aoi_rescaled/')

In [13]:
# For each of the LAI images, we rescale the Geotiff
path = 'C:/Users/mieke/Documents/Data_download/GEE/SSM/Telangana/img_rescaled/'
file_names = os.listdir(path) # contains all filenames
for j in file_names:
    tiff_path = path + j # the selected GeoTIFF path
    functions_preprocessing.crop_image(j, tiff_path, bound_Telangana, 'C:/Users/mieke/Documents/Data_download/GEE/SSM/Telangana/aoi_rescaled/')

In [21]:
# For each of the LAI images, we rescale the Geotiff
path = 'C:/Users/mieke/Documents/Data_download/GEE/SUSM/Telangana/img_rescaled/'
file_names = os.listdir(path) # contains all filenames
for j in file_names:
    tiff_path = path + j # the selected GeoTIFF path
    functions_preprocessing.crop_image(j, tiff_path, bound_Telangana, 'C:/Users/mieke/Documents/Data_download/GEE/SUSM/Telangana/aoi_rescaled/')

In [22]:
# For each of the LAI images, we rescale the Geotiff
path = 'C:/Users/mieke/Documents/Data_download/GEE/NDVI/Telangana/img_rescaled/'
file_names = os.listdir(path) # contains all filenames
for j in file_names:
    tiff_path = path + j # the selected GeoTIFF path
    functions_preprocessing.crop_image(j, tiff_path, bound_Telangana, 'C:/Users/mieke/Documents/Data_download/GEE/NDVI/Telangana/aoi_rescaled/')

In [23]:
# For each of the LAI images, we rescale the Geotiff
path = 'C:/Users/mieke/Documents/Data_download/GEE/LST/Telangana/img_rescaled/'
file_names = os.listdir(path) # contains all filenames
for j in file_names:
    tiff_path = path + j # the selected GeoTIFF path
    functions_preprocessing.crop_image(j, tiff_path, bound_Telangana, 'C:/Users/mieke/Documents/Data_download/GEE/LST/Telangana/aoi_rescaled/')

## We should run the following in the OSGeo4W Shell for Cloud Optimized GeoTIFs
for %N in (C:/Users/mieke/Documents/Data_download/Copernicus/LAI/Telangana/aoi_rescaled/*.tif) DO gdal_translate C:/Users/mieke/Documents/Data_download/Copernicus/LAI/Telangana/aoi_rescaled/%~nN.tif  C:/Users/mieke/Documents/Data_download/Copernicus/LAI/Telangana/cog/%~nN.tif -co COMPRESS=LZW -co TILED=YES

for %N in (C:/Users/mieke/Documents/Data_download/GEE/SSM/Telangana/aoi_rescaled/*.tif) DO gdal_translate C:/Users/mieke/Documents/Data_download/GEE/SSM/Telangana/aoi_rescaled/%~nN.tif  C:/Users/mieke/Documents/Data_download/GEE/SSM/Telangana/cog/%~nN.tif -co COMPRESS=LZW -co TILED=YES

for %N in (C:/Users/mieke/Documents/Data_download/GEE/SUSM/Telangana/aoi_rescaled/*.tif) DO gdal_translate C:/Users/mieke/Documents/Data_download/GEE/SUSM/Telangana/aoi_rescaled/%~nN.tif  C:/Users/mieke/Documents/Data_download/GEE/SUSM/Telangana/cog/%~nN.tif -co COMPRESS=LZW -co TILED=YES

for %N in (C:/Users/mieke/Documents/Data_download/GEE/NDVI/Telangana/aoi_rescaled/*.tif) DO gdal_translate C:/Users/mieke/Documents/Data_download/GEE/NDVI/Telangana/aoi_rescaled/%~nN.tif  C:/Users/mieke/Documents/Data_download/GEE/NDVI/Telangana/cog/%~nN.tif -co COMPRESS=LZW -co TILED=YES

for %N in (C:/Users/mieke/Documents/Data_download/GEE/LST/Telangana/aoi_rescaled/*.tif) DO gdal_translate C:/Users/mieke/Documents/Data_download/GEE/LST/Telangana/aoi_rescaled/%~nN.tif  C:/Users/mieke/Documents/Data_download/GEE/LST/Telangana/cog/%~nN.tif -co COMPRESS=LZW -co TILED=YES

In [ ]:
# Ambarish wants the layers in the format dd-mm-yyyy
# We do this for every parameter. Note: Change the numbers as shown below such that we derive the correct format
path = 'C:/Users/mieke/Documents/Ambarish/LST_cog/'
file_names = os.listdir(path)
for i in file_names:
    os.rename(path + i, path + i[12:14] + '-' + i[9:11] + '-' + i[4:8] + '.tif')

# i[12:14] + '-' + i[9:11] + '-' + i[4:8] LAI, SSM, LST
# i[13:15] + '-' + i[10:12] + '-' + i[5:9] SUSM, NDVI

## Below, we can check whether the nodatavalue is changed correctly

In [ ]:
#data = r'C:\Users\mieke\Documents\Data_download\Copernicus\LAI\Telangana\aoi_rescaled\LAI_2014-01-31.tif'
#data = r'C:\Users\mieke\Documents\Data_download\GEE\SSM\Telangana\aoi_rescaled\SSM_2015-04-14.tif'
#data = r'C:\Users\mieke\Documents\Data_download\GEE\SUSM\Telangana\aoi_rescaled\SUSM_2015-04-14.tif'
#data = r'C:\Users\mieke\Documents\Data_download\GEE\NDVI\Telangana\aoi_rescaled\NDVI_2000-03-05.tif'
#data = r'C:\Users\mieke\Documents\Data_download\GEE\LST\Telangana\aoi_rescaled\LST_2000-02-28.tif'
parameter = rasterio.open(data, mode = 'r')

# Assign raster values to a numpy nd array
param_array = parameter.read(1) # landuse corresponding to each rasterpixel, so we extracted the pixel values from the raster
# affine: 1: corresponds to the width of each pixel, 2: row rotation, 3: x-coordinate of the upper left pixel, 4: column rotation, 5: height of each pixel, 6: y-coordinate of the upper left pixel
#affine = parameter.transform
#print(parameter.meta)
# Geographical coordinate reference system is used (4326)
print(parameter.nodata) # value which has been assigned for the nodata

In [6]:
#data = r'C:\Users\mieke\Documents\Data_download\Copernicus\LAI\Telangana\aoi_rescaled\LAI_2014-01-31.tif'
#data = r'C:\Users\mieke\Documents\Data_download\GEE\SSM\Telangana\aoi_rescaled\SSM_2015-04-14.tif'
#data = r'C:\Users\mieke\Documents\Data_download\GEE\SUSM\Telangana\aoi_rescaled\SUSM_2015-04-14.tif'
#data = r'C:\Users\mieke\Documents\Data_download\GEE\NDVI\Telangana\aoi_rescaled\NDVI_2000-03-05.tif'
#data = r'C:\Users\mieke\Documents\Data_download\GEE\LST\Telangana\aoi_rescaled\LST_2000-02-28.tif'

tiff_open = gdal.Open(data) # open the GeoTIFF we want to rescale
band = tiff_open.GetRasterBand(1) # Select the band
tiff_array = tiff_open.ReadAsArray() # Assign raster values to a numpy nd array
#tiff_open.GetRasterBand(1).SetNoDataValue(-9999)
nodata_value = tiff_open.GetRasterBand(1).GetNoDataValue() # value which has been assigned for the nodata
print(nodata_value)

-9999.0
